<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 3 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [19]:
### YOUR CODE STARTS HEREimport pandas as pd
import pandas as pd
import numpy as np
import seaborn as sns
from scipy.stats import ttest_ind, ttest_ind_from_stats, ttest_rel

### 1) LOAD AND CLEAN THE DATA ###
df= pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data', 
names = ['Party', 'Vote 1', 'Vote 2', 'Vote 3','Vote 4','Vote 5','Vote 6','Vote 7','Vote 8','Vote 9',
                            'Vote 10','Vote 11','Vote 12','Vote 13','Vote 14','Vote 15', 'Vote 16'])
df.head()

,Party,Vote 1,Vote 2,Vote 3,Vote 4,Vote 5,Vote 6,Vote 7,Vote 8,Vote 9,Vote 10,Vote 11,Vote 12,Vote 13,Vote 14,Vote 15,Vote 16
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


In [21]:
#Change letters to numbers. Not sure if ?s are missing values, neutral votes, or congressor refused to vote. Can I just assign them as 0s?
df.replace(to_replace = 'y', value = 1, inplace=True)
df.replace(to_replace = 'n', value = -1, inplace=True)
df.replace(to_replace = '?', value = 0, inplace=True)
df.head()

,Party,Vote 1,Vote 2,Vote 3,Vote 4,Vote 5,Vote 6,Vote 7,Vote 8,Vote 9,Vote 10,Vote 11,Vote 12,Vote 13,Vote 14,Vote 15,Vote 16
0,republican,-1,1,-1,1,1,1,-1,-1,-1,1,0,1,1,1,-1,1
1,republican,-1,1,-1,1,1,1,-1,-1,-1,-1,-1,1,1,1,-1,0
2,democrat,0,1,1,0,1,1,-1,-1,-1,-1,1,-1,1,1,-1,-1
3,democrat,-1,1,1,-1,0,1,-1,-1,-1,-1,1,-1,1,-1,-1,1
4,democrat,1,1,1,-1,1,1,-1,-1,-1,-1,1,0,1,1,1,1


In [22]:
### 2) Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01 ###
dem = df.loc[df['Party']=='democrat']
rpb = df.loc[df['Party']=='republican']

In [23]:
dem.head()

,Party,Vote 1,Vote 2,Vote 3,Vote 4,Vote 5,Vote 6,Vote 7,Vote 8,Vote 9,Vote 10,Vote 11,Vote 12,Vote 13,Vote 14,Vote 15,Vote 16
2,democrat,0,1,1,0,1,1,-1,-1,-1,-1,1,-1,1,1,-1,-1
3,democrat,-1,1,1,-1,0,1,-1,-1,-1,-1,1,-1,1,-1,-1,1
4,democrat,1,1,1,-1,1,1,-1,-1,-1,-1,1,0,1,1,1,1
5,democrat,-1,1,1,-1,1,1,-1,-1,-1,-1,-1,-1,1,1,1,1
6,democrat,-1,1,-1,1,1,1,-1,-1,-1,-1,-1,-1,0,1,1,1


In [24]:
rpb.head() #based on these five values, looks like Vote 3 is favored by dems

,Party,Vote 1,Vote 2,Vote 3,Vote 4,Vote 5,Vote 6,Vote 7,Vote 8,Vote 9,Vote 10,Vote 11,Vote 12,Vote 13,Vote 14,Vote 15,Vote 16
0,republican,-1,1,-1,1,1,1,-1,-1,-1,1,0,1,1,1,-1,1
1,republican,-1,1,-1,1,1,1,-1,-1,-1,-1,-1,1,1,1,-1,0
7,republican,-1,1,-1,1,1,1,-1,-1,-1,-1,-1,-1,1,1,0,1
8,republican,-1,1,-1,1,1,1,-1,-1,-1,-1,-1,1,1,1,-1,1
10,republican,-1,1,-1,1,1,-1,-1,-1,-1,-1,0,0,1,1,-1,-1


In [33]:
mean_votes = pd.DataFrame({'Democrats': dem.mean(), 'Republicans': rpb.mean()})
mean_votes

,Democrats,Republicans
Party,0j,NaN
Vote 1,(0.20224719101123595+0j),-0.613095
Vote 10,(-0.056179775280898875+0j),0.113095
Vote 11,(0.011235955056179775+0j),-0.696429
Vote 12,(-0.6629213483146067+0j),0.684524
Vote 13,(-0.3970037453183521+0j),0.678571
Vote 14,(-0.2883895131086142+0j),0.922619
Vote 15,(0.25842696629213485+0j),-0.761905
Vote 16,(0.602996254681648+0j),0.273810
Vote 2,(0.003745318352059925+0j),0.011905


In [35]:
stat, pvalue = ttest_ind(dem, rpb)
pvalue <= 0.05


C:\Users\Kevona Jean\myCODE\lib\site-packages\scipy\stats\stats.py:245: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


TypeError: unsupported operand type(s) for /: 'str' and 'int'